In [1]:
import itertools

In [72]:
import numpy as np, pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [123]:
def heuristic(C):
    S = C.copy()
    S.loc[:, :] = -np.infty
    for i, j in itertools.product(C.index, C.columns):
        if C.loc[i,j].round(2) != 0:
            S.loc[i,j] = -np.infty
            continue
        mr, mc = 2*[np.infty]
        for i1 in C.index:
            if i1 == i: continue
            mc = min(mc, C.loc[i1, j])
        for j1 in C.columns:
            if j1 == j: continue
            mr = min(mr, C.loc[i, j1])
        S.loc[i,j] = mr + mc
    return S

In [118]:
def row_transform(C):
    m = C.min(axis=1)
    return C.sub(m, axis='index'), m

def col_transform(C):
    m = C.min(axis=0)
    return C.sub(m), m

In [92]:
N=6
C = np.array([
    [7+5*np.abs(np.sin(i+1)*np.cos(1+j)) for j in range(N)]
    for i in range(N)
])
C = C.round(2)
C = pd.DataFrame(C, index=[1+i for i in range(N)], columns=[1+i for i in range(N)])
C

,1,2,3,4,5,6
1,9.27,8.75,11.17,9.75,8.19,11.04
2,9.46,8.89,11.50,9.97,8.29,11.37
3,7.38,7.29,7.70,7.46,7.20,7.68
4,9.04,8.57,10.75,9.47,8.07,10.63
5,9.59,9.00,11.75,10.13,8.36,11.60
6,7.75,7.58,8.38,7.91,7.40,8.34


In [93]:
for i in range(N):
    C.loc[1+i, 1+i] = np.infty
C

,1,2,3,4,5,6
1,inf,8.75,11.17,9.75,8.19,11.04
2,9.46,inf,11.50,9.97,8.29,11.37
3,7.38,7.29,inf,7.46,7.20,7.68
4,9.04,8.57,10.75,inf,8.07,10.63
5,9.59,9.00,11.75,10.13,inf,11.60
6,7.75,7.58,8.38,7.91,7.40,inf


In [115]:
C2, C2mr = row_transform(C)
C2 = C2.round(2)
C2

,1,2,3,4,5,6
1,inf,0.56,2.98,1.56,0.00,2.85
2,1.17,inf,3.21,1.68,0.00,3.08
3,0.18,0.09,inf,0.26,0.00,0.48
4,0.97,0.50,2.68,inf,0.00,2.56
5,0.59,0.00,2.75,1.13,inf,2.60
6,0.35,0.18,0.98,0.51,0.00,inf


In [116]:
C2mr

1   8.19
2   8.29
3   7.20
4   8.07
5   9.00
6   7.40
dtype: float64

In [119]:
C2, C2mc = col_transform(C2)
C2 = C2.round(2)
C2

,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,2.37
2,0.99,inf,2.23,1.42,0.00,2.60
3,0.00,0.09,inf,0.00,0.00,0.00
4,0.79,0.50,1.70,inf,0.00,2.08
5,0.41,0.00,1.77,0.87,inf,2.12
6,0.17,0.18,0.00,0.25,0.00,inf


In [120]:
C2mc

1   0.18
2   0.00
3   0.98
4   0.26
5   0.00
6   0.48
dtype: float64

In [122]:
l2r, l2c = C2mr.sum(), C2mc.sum()
l2 = l2r + l2c
l2r, l2c, l2

(48.15, 1.9, 50.05)

In [124]:
H2 = heuristic(C2)
H2

,1,2,3,4,5,6
1,-inf,-inf,-inf,-inf,0.56,-inf
2,-inf,-inf,-inf,-inf,0.99,-inf
3,0.17,-inf,-inf,0.25,0.00,2.08
4,-inf,-inf,-inf,-inf,0.50,-inf
5,-inf,0.50,-inf,-inf,-inf,-inf
6,-inf,-inf,1.70,-inf,0.00,-inf


In [125]:
B1u = 8.19+9+9.97+10.63+8.38+7.38
B1u, round(B1u, 2)

(53.550000000000004, 53.55)

In [126]:
C2

,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,2.37
2,0.99,inf,2.23,1.42,0.00,2.60
3,0.00,0.09,inf,0.00,0.00,0.00
4,0.79,0.50,1.70,inf,0.00,2.08
5,0.41,0.00,1.77,0.87,inf,2.12
6,0.17,0.18,0.00,0.25,0.00,inf


In [127]:
round(50.05+2.08, 2)

52.13

In [129]:
C_2_1 = C2.loc[[1,2,4,5,6], [1, 2, 3, 4, 5]]
# C_2_1 = C_2_1[:,np.array([1,2,3,4,5])-1]
C_2_1.loc[6, 3] = np.infty
C_2_1

,1,2,3,4,5
1,inf,0.56,2.00,1.30,0.00
2,0.99,inf,2.23,1.42,0.00
4,0.79,0.50,1.70,inf,0.00
5,0.41,0.00,1.77,0.87,inf
6,0.17,0.18,inf,0.25,0.00


In [130]:
C3, C3_mr = row_transform(C_2_1)
C3

,1,2,3,4,5
1,inf,0.56,2.00,1.30,0.00
2,0.99,inf,2.23,1.42,0.00
4,0.79,0.50,1.70,inf,0.00
5,0.41,0.00,1.77,0.87,inf
6,0.17,0.18,inf,0.25,0.00


In [131]:
C3_mr

1   0.00
2   0.00
4   0.00
5   0.00
6   0.00
dtype: float64

In [132]:
C3, C3_mc = col_transform(C3)
C3

,1,2,3,4,5
1,inf,0.56,0.30,1.05,0.00
2,0.82,inf,0.53,1.17,0.00
4,0.62,0.50,0.00,inf,0.00
5,0.24,0.00,0.07,0.62,inf
6,0.00,0.18,inf,0.00,0.00


In [133]:
C3_mc

1   0.17
2   0.00
3   1.70
4   0.25
5   0.00
dtype: float64

In [135]:
l3 = C3_mr.sum() + C3_mc.sum()
l3

2.12

In [136]:
round(l2 + l3, 2)

52.17

In [137]:
H3 = heuristic(C3)
H3

,1,2,3,4,5
1,-inf,-inf,-inf,-inf,0.30
2,-inf,-inf,-inf,-inf,0.53
4,-inf,-inf,0.07,-inf,0.00
5,-inf,0.25,-inf,-inf,-inf
6,0.24,-inf,-inf,0.62,0.00


In [138]:
52.17+.53

52.7

In [139]:
C4 = C3.loc[[1,4,5,6,], [1,2,3,4]]
C4.loc[5, 2] = np.infty
C4

,1,2,3,4
1,inf,0.56,0.30,1.05
4,0.62,0.50,0.00,inf
5,0.24,inf,0.07,0.62
6,0.00,0.18,inf,0.00


In [140]:
C4, C4mr = row_transform(C4)
print(C4mr)
C4

1   0.30
4   0.00
5   0.07
6   0.00
dtype: float64


,1,2,3,4
1,inf,0.26,0.00,0.75
4,0.62,0.50,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.18,inf,0.00


In [141]:
C4, C4mc = col_transform(C4)
print(C4mc)
C4

1   0.00
2   0.18
3   0.00
4   0.00
dtype: float64


,1,2,3,4
1,inf,0.08,0.00,0.75
4,0.62,0.32,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.00,inf,0.00


In [142]:
H4 = heuristic(C4)
H4

,1,2,3,4
1,-inf,-inf,0.08,-inf
4,-inf,-inf,0.32,-inf
5,-inf,-inf,0.17,-inf
6,0.17,0.08,-inf,0.55


In [143]:
L4 = C4mr.sum() + C4mc.sum()
L4

0.55

In [144]:
L4 + l3 + l2

52.72

In [145]:
round(52.72+.55, 2)

53.27

In [146]:
C5 = C4.loc[[1,4,5], [1,2,3]]
C5

,1,2,3
1,inf,0.08,0.00
4,0.62,0.32,0.00
5,0.17,inf,0.00


In [147]:
C5, C5mc = col_transform(C5)
C5

,1,2,3
1,inf,0.00,0.00
4,0.45,0.24,0.00
5,0.00,inf,0.00


In [148]:
C5mc.sum(), C5mc

(0.24999999999999992, 1   0.17
 2   0.08
 3   0.00
 dtype: float64)

In [149]:
L5 = C5mc.sum()
round(L5 + L4 + l3 + l2, 2)

52.97

In [150]:
H5 = heuristic(C5)
H5

,1,2,3
1,-inf,0.24,0.00
4,-inf,-inf,0.24
5,0.45,-inf,0.00


In [151]:
52.97 + .45

53.42

In [152]:
52.97+.32

53.29

In [153]:
C7 = C4.copy()
C7

,1,2,3,4
1,inf,0.08,0.00,0.75
4,0.62,0.32,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.00,inf,0.00
